# Capstone Week 3 Peer Graded Assignment

            Segmenting and Clustering Neighborhoods in Toronto

In [181]:
# import libraries 
import pandas as pd
import numpy as np
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup

import requests # library to handle requests

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')

Libraries imported.


-All the magic is done with beautifulsoup 
-First you read the FIRST table in the wiki page

-you can find an example here : https://stackoverflow.com/questions/48372543/beautifulsoup-scraping-table-from-website-with-requests-for-pandas

link for reference
 video
 https://cognitiveclass.ai/blog/event/exploring-neighborhoods-using-clustering-and-foursquare/?instance_id=242
 
 https://www.youtube.com/watch?v=ng2o98k983k

In [182]:
# use requests to get the link in html format 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(url)
#res

In [3]:
# use soup to start the scraping process
soup = BeautifulSoup(res.content,'lxml')
#soup

-The data in the wiki page you need to scrape is enclosed in what is called an HTML table
-begining by <table..> ending by </table>

In [6]:
# find the data or table with all the information
table = soup.find_all('table')[0]
#table

In [94]:
# read the html table in pandas form 
df = pd.read_html(str(table))[0]
df.head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park Ontario Provincial Government,M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


Once you've read the HTML table into a pandas dataframe, you'll see it's not exactly the structure you need in the assignment

-but all the values are there!  

-You'll see also that you're interested in the CELL contents.. each cell represents a line in your FINAL dataframe

-the way I did it... through a functon called flatten()

-mydataframe.values.flatten().tolist()

-I'm interested in only the values, which I flatten (that means only ONE column), then I convert it into a list

-this list contains now all the values I am interested in...

In [113]:
list = df.values.flatten().tolist()
#list

In [74]:
print(df.shape)
type(df)

(20, 9)


pandas.core.frame.DataFrame

-Now begins the parsing (or wrangling..)

-NEXT STEP : delete all the "Not assigned" lines

-I build a new list from the one I already have.. by excluding the values containing the term 'Not assigned'

-something like :
-newlist = [  s for s in oldlist if "Not assigned" not in s]

-now newlist contains strings of the sort :
-M3ANorth York(Parkwoods)

-clearly : the first 3 characters are the PostalCode, the string from index 3 all the way to first (, is the Borough column

-and the substring inside the (...) is the neighborhood

-I'll let you do these steps... if you have a difficulty with the last steps, I'll give another hint

In [186]:
# creat a new list without "Not assigned" values
newlist = [s for s in list if 'Not assigned' not in s]
newlist

['M3ANorth York(Parkwoods)',
 'M4ANorth York(Victoria Village)',
 'M5ADowntown Toronto(Regent Park / Harbourfront)',
 'M6ANorth York(Lawrence Manor / Lawrence Heights)',
 "M7AQueen's Park Ontario Provincial Government",
 'M9AEtobicoke(Islington Avenue)',
 'M1BScarborough(Malvern / Rouge)',
 'M3BNorth York(Don Mills)North',
 'M4BEast York(Parkview Hill / Woodbine Gardens)',
 'M5BDowntown Toronto(Garden District, Ryerson)',
 'M6BNorth York(Glencairn)',
 'M9BEtobicoke(West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale)',
 'M1CScarborough(Rouge Hill / Port Union / Highland Creek)',
 'M3CNorth York(Don Mills)South(Flemingdon Park)',
 'M4CEast York(Woodbine Heights)',
 'M5CDowntown Toronto(St. James Town)',
 'M6CYork(Humewood-Cedarvale)',
 'M9CEtobicoke(Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood)',
 'M1EScarborough(Guildwood / Morningside / West Hill)',
 'M4EEast Toronto(The Beaches)',
 'M5EDowntown Toronto(Berczy Park)',
 'M6EYork(Caledonia-Fair

1°) You build an empty dataframe wtih the 3 columns 'PostalCode', 'Borough', 'Neighborhood'

2°)You loop through the list (103 times) with a counter i for example, you increment i each time

3°) in every execution of the loop, you fill a new row in the dataframe.. using string parsing

look for string slicing

value of the 1st column is indexes 0 through 2

value for column 2 is the substring from index 3 to the index of the first (

In [80]:
len(newlist)

103

In [96]:
print(newlist[0][0:3]) # so here i can replace the [0] with [i]

M3A


In [98]:
print(newlist[0][3:])

North York(Parkwoods)


In [18]:
col2_strt = newlist[0][3:]
col2_strt

'North York(Parkwoods)'

In [43]:
col2_fin = newlist[0].find('(')
col2_fin

13

In [68]:
# now i need to replace all / with , 
test = newlist[2].replace('/',',')
test

'M5ADowntown Toronto(Regent Park , Harbourfront)'

In [167]:
# define data frame columns
column_names = ['Postalcode', 'Borough', 'Neighborhood']

Toronto = pd.DataFrame(columns=column_names)
Toronto

,Postalcode,Borough,Neighborhood


In [168]:
# value of the 1st column is indexes 0 through 2 the first 3 characters are the PostalCode
# the string from index 3 all the way to first (, is the Borough column 
# value for column 2 is the substring from index 3 to the index of the first (
# and the substring inside the (...) is the neighborhood

n = len(newlist)
i = 0
for i in range(n):
    col_postalcode = newlist[i][0:3]    
    col2_temp = newlist[i].find('(')
    col_borough = newlist[i][3:col2_temp] 
    col_neighborhood = newlist[i][col2_temp+1:].replace('/',',').strip(')')
    
    #print(col_postalcode)
    #print(col_borough)
    #print(col_neighborhood)
    #print()
    
    
    #Toronto.loc[i] = [col_postalcode, col_borough, col_neighborhood]
    
    Toronto = Toronto.append({'Postalcode': col_postalcode,
                                          'Borough': col_borough,
                                          'Neighborhood': col_neighborhood} ,ignore_index=True)
                                           




In [156]:
Toronto.head(10)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,(Don Mills) North
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [173]:
# Fixing the dataframe 
Toronto.loc[4][1] = Toronto.loc[4][2] = "Queen's Park"
Toronto.loc[7][2] = "Don Mills North"
Toronto.loc[13][2] = "Don Mills South (Flemingdon Park)"
Toronto.loc[40][2] = "Downsview East (CFB Toronto)"
Toronto.loc[46][2] = "Downsview West"
Toronto.loc[53][2] = "Downsview Central"
Toronto.loc[59][2] = "Willowdale South"
Toronto.loc[60][2] = "Downsview Northwest"
Toronto.loc[72][2] = "Willowdale West"


In [174]:
Toronto

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [ ]:
# The code was removed by Watson Studio for sharing.

In [188]:
Toronto.shape

(103, 3)

In [ ]:
# The code was removed by Watson Studio for sharing.